# 1-Endmembers: MELTS Liquid Components Standard State Code Generator

# 1-Endmembers: MELTS Liquid Components Standard State Code Generator

This notebook generates code to implement endmember component thermodynamic properties for a rhyolite-MELTS liquid model extended to include CO<sub>2</sub> and ZrSiO<sub>4</sub>

after:  
**Ghiorso MS, Sack RO (1995) Chemical Mass Transfer in Magmatic Processes IV. A revised and internally consistent thermodynamic model for the interpolation and extrapolation of liquid-solid equilibria in magmatic systems at elevated temperatures and pressures. Contrib Mineral Petrology 119, 197-212**  
and  
**Gualda, G.A.R., Ghiorso, M.S., Lemons, R.V., Carley, T.L. (2012) Rhyolite-MELTS: A modified calibration of MELTS optimized for silica-rich, fluid-bearing magmatic systems. Journal of Petrology, 53, 875-890**  
and  
**Ghiorso, M.S., Gualda, G.A.R. (2015) An H2O-CO2 mixed fluid saturation model compatible with rhyolite-MELTS. Contributions to Mineralogy and Petrology, doi:10.1007/s00410-015-1141-8**

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import math
sym.init_printing()
from thermoengine import coder,phases
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Special case SiO<sub>2</sub>

In [ ]:
model = coder.StdStateModel()
model.set_module_name('rMELTS_ZR')
model_type = "calib"
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

G(liquid) T<sub>r</sub>, P<sub>r</sub> -\> G(liquid) T<sub>g</sub>, P<sub>r</sub>

In [ ]:
h0,s0 = sym.symbols('h0 s0')
params = [('h0','J',h0), ('s0','J/K', s0)]
GPr = h0 - T*s0
model.add_expression_to_model(GPr, params)

In [ ]:
al,bl,cl,dl,tg = sym.symbols('al bl cl dl tg')
params = [('al', 'J/K', al), ('bl', 'J/K^2', bl), ('cl', 'J-K^2', cl), ('dl', 'J-K^1/2', dl), ('tg', 'K', tg)]
Cpg = al + bl*T + cl/T**2 + dl/sym.sqrt(T)
GPr = sym.integrate(Cpg,(T,Tr,T)) - T*sym.integrate(Cpg/T,(T,Tr,T))
model.add_expression_to_model(GPr, params, exp_type='restricted', lower_limits=(Tr,None), upper_limits=(tg,None))

In [ ]:
dummy = sym.symbols('dummy')
params = [('dummy', 'J', dummy)]
model.add_expression_to_model(GPr.subs(T,tg), params, exp_type='restricted', lower_limits=(tg,None), upper_limits=(None,None))

In [ ]:
cpl = sym.symbols('cpl')
params = [('cpl', 'J/K', cpl)]
GPr = sym.integrate(cpl,(T,tg,T)) - T*sym.integrate(cpl/T,(T,tg,T))
model.add_expression_to_model(GPr, params, exp_type='restricted', lower_limits=(tg,None), upper_limits=(None,None))

In [ ]:
v,dvdt,dvdp,d2vdtdp,d2vdp2,trl = sym.symbols('v dvdt dvdp d2vdtdp d2vdp2 trl')
params = [('v', 'J/bar', v), ('dvdt', 'J/bar-K', dvdt), ('dvdp', 'J/bar^2', dvdp), ('d2vdtdp', 'J/bar^2-K', d2vdtdp), 
          ('d2vdp2', 'J/bar^3', d2vdp2), ('trl','K',trl)]
GPrP = sym.integrate(v + dvdt*(T-trl) + (dvdp + d2vdtdp*(T-trl))*(P-Pr) + d2vdp2*(0.5*P**2 - Pr*(P-Pr)),(P,Pr,P))
model.add_expression_to_model(GPrP, params)

### Build instance

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

Parameters from Ghiorso and Sack (1995)

In [ ]:
param_dict = {
    'Phase': 'SiO2',
    'Formula': 'Si(1)O(2)',
    'h0':   -901554.0,
    's0':        48.475,
    'al':       127.200,
    'bl':       -10.777e-3,
    'cl':         4.3127e5,
    'dl':     -1463.8,
    'tg':      1480.0,
    'cpl':       81.373,
    'v':          2.690,
    'dvdt':       0.0,
    'dvdp':      -1.89e-5,
    'd2vdtdp':    1.3e-8,
    'd2vdp2':     3.6e-10, 
    'T_r':      298.15,
    'P_r':        1.0,
    'trl':     1673.15,
    'dummy':      1.0
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cp rMELTS_ZR.pyx endmembers.pyx
endmember_file_list = ['SiO2_rMELTS_ZR_calib.c']
endmember_name_list = ['SiO2']

## General case (Berman solids -\> liquid)

In [ ]:
model = coder.StdStateModel()
model.set_module_name('rMELTS_ZR')
model_type = "calib"
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

G(solid) at T, P<sub>r</sub>: lattice heat capacity  
G(Solid) -\> G(liquid)

In [ ]:
k0,k1,k2,k3,tFusion,sFusion,cpLiq = sym.symbols('k0 k1 k2 k3 tFusion sFusion cpLiq')
CpPr = k0 + k1/sym.sqrt(T) + k2/T**2 + k3/T**3
STrPr,HTrPr = sym.symbols('S_TrPr H_TrPr')
params = [('H_TrPr','J',HTrPr), ('S_TrPr','J/K',STrPr), ('k0','J/K-m',k0), ('k1','J-K^(1/2)-m',k1),
          ('k2','J-K/m',k2),  ('k3','J-K^2',k3), ('tFusion','J',tFusion), ('sFusion','J/K', sFusion), ('cpLiq', 'J/K', cpLiq)]
GPr = HTrPr + sym.integrate(CpPr,(T,Tr,tFusion)) - T*(STrPr + sym.integrate(CpPr/T,(T,Tr,tFusion))) \
    + sFusion*tFusion + cpLiq*(T-tFusion) - T*(sFusion + cpLiq*sym.log(T/tFusion))
model.add_expression_to_model(GPr, params)

In [ ]:
GPr

G(solid) at T, P<sub>r</sub>: lambda transition

In [ ]:
l1,l2 = sym.symbols('l1 l2')
Tl, Tlr = sym.symbols('T_lambda_Pr T_lambda_ref')
params = [('l1','(J/m)^(1/2)-K', l1), ('l2', '(J/m)^(1/2)/K^2', l2), ('T_lambda_Pr', 'K', Tl), ('T_lambda_ref', 'K', Tlr)]
Cpl = T*(l1+l2*T)**2
Gl = sym.integrate(Cpl,(T,Tlr,Tl)) - T*sym.integrate(Cpl/T,(T,Tlr,Tl))
model.add_expression_to_model(Gl, params, exp_type='restricted', lower_limits=(Tr,None), upper_limits=(None,None))

G(solid) at T, P<sub>r</sub>: first order phase transition

In [ ]:
deltaHt = sym.symbols('H_t')
params = [('H_t','J/m', deltaHt)]
GaboveTl = -(T-Tl)*deltaHt/Tl
model.add_expression_to_model(GaboveTl , params, exp_type='restricted', lower_limits=(Tr,None), upper_limits=(None,None))

G(liquid) at T, P<sub>r</sub> -\> G(liquid) at T, P

In [ ]:
Vtrl,dvdt,dvdp,d2vdtdp,d2vdp2,trl = sym.symbols('Vtrl dvdt dvdp d2vdtdp d2vdp2 trl')
params = [('Vtrl', 'J/bar', Vtrl), ('dvdt','J/bar-K',dvdt), ('dvdp','J/bar^2',dvdp), 
          ('d2vdtdp','J/bar^2-K',d2vdtdp),  ('d2vdp2','J/bar^3',d2vdp2), ('trl', 'K', trl)]
GTPrToP = (Vtrl + dvdt*(T-trl))*(P-Pr) + (dvdp + (T-trl)*d2vdtdp)*(P*P-Pr*Pr)/2 - (dvdp + (T-trl)*d2vdtdp)*Pr*(P-Pr) \
        + d2vdp2*( (P*P*P-Pr*Pr*Pr)/6 - Pr*(P*P-Pr*Pr)/2 + Pr*Pr*(P-Pr)/2 )
model.add_expression_to_model(GTPrToP, params)

In [ ]:
GTPrToP

### Build instances

Parameters from Ghiorso and Sack (1995), unless noted.

#### TiO<sub>2</sub>

In [ ]:
param_dict = {
    'Phase': 'TiO2',
    'Formula': 'Ti(1)O(2)',
    'H_TrPr':   -944750.0,
    'S_TrPr':        50.460,
    'k0':            77.84,
    'k1':             0.0,
    'k2':           -33.678e5,
    'k3':            40.294e7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           2.316,
    'dvdt':           7.246e-4,
    'dvdp':          -2.310e-5,
    'd2vdtdp':        0.0,
    'd2vdp2':         5.0e-10, 
    'tFusion':     1870.0,
    'sFusion':       35.824,
    'cpLiq':        109.2,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('TiO2_rMELTS_ZR_calib.c')
endmember_name_list.append('TiO2')

#### Al<sub>2</sub>O<sub>3</sub>

In [ ]:
param_dict = {
    'Phase': 'Al2O3',
    'Formula': 'Al(2)O(3)',
    'H_TrPr':  -1675700.0,
    'S_TrPr':        50.82,
    'k0':           155.02,
    'k1':            -8.284E2,
    'k2':           -38.614E5,
    'k3':           40.908E7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           3.711,
    'dvdt':           2.62e-4,
    'dvdp':          -2.26e-5,
    'd2vdtdp':        2.7e-8,
    'd2vdp2':         4.0e-10, 
    'tFusion':     2319.65,
    'sFusion':       48.61,
    'cpLiq':        170.3,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Al2O3_rMELTS_ZR_calib.c')
endmember_name_list.append('Al2O3')

#### Fe<sub>2</sub>O<sub>3</sub>

In [ ]:
param_dict = {
    'Phase': 'Fe2O3',
    'Formula': 'Fe(2)O(3)',
    'H_TrPr':   -822000.0,
    'S_TrPr':        87.40,
    'k0':           146.86,
    'k1':             0.0,
    'k2':           -55.768E5,
    'k3':            52.563E7,
    'l1':            -7.403E-2,
    'l2':            27.921E-5,
    'T_lambda_Pr':  955.0,
    'T_lambda_ref': 298.15,
    'H_t':         1287.0,
    'Vtrl':           4.213,
    'dvdt':           9.09E-4,
    'dvdp':          -2.53E-5,
    'd2vdtdp':        3.1E-8,
    'd2vdp2':         4.4e-10, 
    'tFusion':     1895.0,
    'sFusion':       60.41,
    'cpLiq':        240.9,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Fe2O3_rMELTS_ZR_calib.c')
endmember_name_list.append('Fe2O3')

#### MgCr<sub>2</sub>O<sub>4</sub>

In [ ]:
param_dict = {
    'Phase': 'MgCr2O4',
    'Formula': 'Mg(1)Cr(2)O(4)',
    'H_TrPr':  -1783640.0,
    'S_TrPr':       106.02,
    'k0':           201.981,
    'k1':            -5.519E2,
    'k2':           -57.844E5,
    'k3':           57.729E7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           5.358,
    'dvdt':          11.71e-4,
    'dvdp':          -2.26e-5,
    'd2vdtdp':        1.8e-8,
    'd2vdp2':         4.67e-10, 
    'tFusion':     2673.15,
    'sFusion':       73.22,
    'cpLiq':        335.1,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('MgCr2O4_rMELTS_ZR_calib.c')
endmember_name_list.append('MgCr2O4')

#### Fe<sub>2</sub>SiO<sub>4</sub>

In [ ]:
param_dict = {
    'Phase': 'Fe2SiO4',
    'Formula': 'Fe(2)Si(1)O(4)',
    'H_TrPr':  -1479360.0,
    'S_TrPr':       150.930,
    'k0':           248.93,
    'k1':           -19.239E2,
    'k2':             0.0,
    'k3':           -13.910E7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           5.420,
    'dvdt':           5.84e-4,
    'dvdp':          -2.79e-5,
    'd2vdtdp':       -2.3e-8,
    'd2vdp2':        14.6e-10, 
    'tFusion':      1490.0,
    'sFusion':       59.9,
    'cpLiq':        240.2,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Fe2SiO4_rMELTS_ZR_calib.c')
endmember_name_list.append('Fe2SiO4')

#### MnSi<sub>0.5</sub>O<sub>2</sub>

In [ ]:
param_dict = {
    'Phase': 'MnSi1d2O2',
    'Formula': 'Mn(1)Si(0.5)O(2)',
    'H_TrPr':  -1732000.0/2.0,
    'S_TrPr':       155.9/2.0,
    'k0':           219.89/2.0,
    'k1':           -12.710E2/2.0,
    'k2':           -20.496E5/2.0,
    'k3':            17.652E7/2.0,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           2.84,
    'dvdt':           5.84e-4/2.0,
    'dvdp':          -2.79e-5/2.0,
    'd2vdtdp':       -2.3e-8/2.0,
    'd2vdp2':        14.6e-10/2.0, 
    'tFusion':      1620.0,
    'sFusion':       27.6,
    'cpLiq':        121.6,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('MnSi1d2O2_rMELTS_ZR_calib.c')
endmember_name_list.append('MnSi1d2O2')

#### Mg<sub>2</sub>SiO<sub>4</sub>

In [ ]:
param_dict = {
    'Phase': 'Mg2SiO4',
    'Formula': 'Mg(2)Si(1)O(4)',
    'H_TrPr':  -2174420.0,
    'S_TrPr':        94.010,
    'k0':           238.64,
    'k1':           -20.013E2,
    'k2':             0.0,
    'k3':           -11.624E7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           4.980,
    'dvdt':           5.24e-4,
    'dvdp':          -1.35e-5,
    'd2vdtdp':       -1.3e-8,
    'd2vdp2':         4.14e-10, 
    'tFusion':      2163.0,
    'sFusion':       57.2,
    'cpLiq':        271.0,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Mg2SiO4_rMELTS_ZR_calib.c')
endmember_name_list.append('Mg2SiO4')

#### NiSi<sub>0.5</sub>O<sub>2</sub>

In [ ]:
param_dict = {
    'Phase': 'NiSi1d2O2',
    'Formula': 'Ni(1)Si(0.5)O(2)',
    'H_TrPr':  -1395300.0/2.0,
    'S_TrPr':       128.1/2.0,
    'k0':           214.997/2.0,
    'k1':           -10.3075E2/2.0,
    'k2':           -49.4453E5/2.0,
    'k3':            62.375E7/2.0,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           2.48,
    'dvdt':           5.84e-4/2.0,
    'dvdp':          -2.79e-5/2.0,
    'd2vdtdp':       -2.3e-8/2.0,
    'd2vdp2':        14.6e-10/2.0, 
    'tFusion':      1923.0,
    'sFusion':       29.0,
    'cpLiq':        119.3,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('NiSi1d2O2_rMELTS_ZR_calib.c')
endmember_name_list.append('NiSi1d2O2')

#### CoSi<sub>0.5</sub>O<sub>2</sub>

In [ ]:
param_dict = {
    'Phase': 'CoSi1d2O2',
    'Formula': 'Co(1)Si(0.5)O(2)',
    'H_TrPr':  -1414100.0/2.0,
    'S_TrPr':       142.6/2.0,
    'k0':           201.048/2.0,
    'k1':            -0.369E2/2.0,
    'k2':           -71.81E5/2.0,
    'k3':            90.05E7/2.0,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           2.30,
    'dvdt':           5.84e-4/2.0,
    'dvdp':          -2.79e-5/2.0,
    'd2vdtdp':       -2.3e-8/2.0,
    'd2vdp2':        14.6e-10/2.0, 
    'tFusion':      1688.0,
    'sFusion':       29.0,
    'cpLiq':        125.3,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('CoSi1d2O2_rMELTS_ZR_calib.c')
endmember_name_list.append('CoSi1d2O2')

#### CaSiO<sub>3</sub>

In [ ]:
param_dict = {
    'Phase': 'CaSiO3',
    'Formula': 'Ca(1)Si(1)O(3)',
    'H_TrPr':  -1627427.0,
    'S_TrPr':        85.279,
    'k0':           141.16,
    'k1':            -4.172e2,
    'k2':           -58.576e5,
    'k3':            94.074e7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           4.347,
    'dvdt':           2.92e-4,
    'dvdp':          -1.55e-5,
    'd2vdtdp':       -1.6e-8,
    'd2vdp2':         3.89e-10, 
    'tFusion':      1817.0,
    'sFusion':       31.5,
    'cpLiq':        172.4,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('CaSiO3_rMELTS_ZR_calib.c')
endmember_name_list.append('CaSiO3')

#### Na<sub>2</sub>SiO<sub>3</sub>

In [ ]:
param_dict = {
    'Phase': 'Na2SiO3',
    'Formula': 'Na(2)Si(1)O(3)',
    'H_TrPr':   -373190.0*4.184,
    'S_TrPr':        27.21*4.184,
    'k0':           234.77,
    'k1':           -22.189E2,
    'k2':             0.0,
    'k3':            13.530E7,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           5.568,
    'dvdt':           7.41e-4,
    'dvdp':          -4.29e-5,
    'd2vdtdp':       -5.3e-8,
    'd2vdp2':         8.4e-10, 
    'tFusion':      1361.0,
    'sFusion':       38.34,
    'cpLiq':        180.2,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Na2SiO3_rMELTS_ZR_calib.c')
endmember_name_list.append('Na2SiO3')

#### KAlSiO<sub>4</sub>

In [ ]:
param_dict = {
    'Phase': 'KAlSiO4',
    'Formula': 'K(1)Al(1)Si(1)O(4)',
    'H_TrPr':  -2111813.55,
    'S_TrPr':       133.9653,
    'k0':           186.0,
    'k1':             0.0,
    'k2':          -131.067E5,
    'k3':           213.893E7,
    'l1':            -7.096454E-2,
    'l2':            21.682E-5,
    'T_lambda_Pr':  800.15,
    'T_lambda_ref': 298.15,
    'H_t':         1154.0,
    'Vtrl':           6.8375,
    'dvdt':           7.265e-4,
    'dvdp':          -6.395e-5,
    'd2vdtdp':       -4.6e-8,
    'd2vdp2':        12.1e-10, 
    'tFusion':      2023.15,
    'sFusion':       24.5,
    'cpLiq':        217.0,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('KAlSiO4_rMELTS_ZR_calib.c')
endmember_name_list.append('KAlSiO4')

#### Ca<sub>3</sub>(PO<sub>3</sub>)<sub>2</sub>

In [ ]:
param_dict = {
    'Phase': 'Ca3P2O8',
    'Formula': 'Ca(3)P(2)O(8)',
    'H_TrPr':  -4097169.0,
    'S_TrPr':       235.978,
    'k0':           402.997,
    'k1':           -28.0835E2,
    'k2':             0.0,
    'k3':           -32.6230E7,
    'l1':             2.5427E-2,
    'l2':            19.255E-5,
    'T_lambda_Pr': 1373.0,
    'T_lambda_ref': 298.15,
    'H_t':        14059.0,
    'Vtrl':          10.7382,
    'dvdt':           0.0,
    'dvdp':           0.0,
    'd2vdtdp':        0.0,
    'd2vdp2':         0.0, 
    'tFusion':      1943.15,
    'sFusion':       35.690,
    'cpLiq':        574.67,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('Ca3P2O8_rMELTS_ZR_calib.c')
endmember_name_list.append('Ca3P2O8')

#### H<sub>2</sub>O

This is a placeholder model for the one described in Ghiorso and Gualda (2015). The real code is copied into the working directory.

In [ ]:
param_dict = {
    'Phase': 'H2O',
    'Formula': 'H(2)O(1)',
    'H_TrPr':    -99999.0,
    'S_TrPr':         0.0,
    'k0':             0.0,
    'k1':             0.0,
    'k2':             0.0,
    'k3':             0.0,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           0.0,
    'dvdt':           0.0,
    'dvdp':           0.0,
    'd2vdtdp':        0.0,
    'd2vdp2':         0.0, 
    'tFusion':     1000.0,
    'sFusion':        0.0,
    'cpLiq':          0.0,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
%cp ../H2O_rMELTS_ZR_calib.c .
%cp ../H2O_rMELTS_ZR_calib.h .
endmember_file_list.append('H2O_rMELTS_ZR_calib.c')
endmember_name_list.append('H2O')

#### CO<sub>2</sub>

The enthalpy, entropy, heat capacity and dCpDt entries are correction terms that should be added to Duan’s pure CO<sub>2</sub>  
This is a placeholder model for the one described in Ghiorso and Gualda (2015). The real code is copied into the working directory.

In [ ]:
param_dict = {
    'Phase': 'CO2',
    'Formula': 'C(1)O(2)',
    'H_TrPr':        -6.3093193811701e-01*1000.0,
    'S_TrPr':        -1.0939331414050e+02,
    'k0':             0.0,
    'k1':             0.0,
    'k2':             0.0,
    'k3':             0.0,
    'l1':             0.0,
    'l2':             0.0,
    'T_lambda_Pr':    0.0,
    'T_lambda_ref':   0.0,
    'H_t':            0.0,
    'Vtrl':           4.0157994267547e+00,
    'dvdt':           1.2131890000000e+00/1000.0,
    'dvdp':          -4.2673870000000e-01/10000.0,
    'd2vdtdp':        0.0,
    'd2vdp2':         0.0, 
    'tFusion':     1000.0,
    'sFusion':        0.0,
    'cpLiq':          0.0,
    'T_r':          298.15,
    'P_r':            1.0,
    'trl':         1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
%cp ../CO2_rMELTS_ZR_calib.c .
%cp ../CO2_rMELTS_ZR_calib.h .
endmember_file_list.append('CO2_rMELTS_ZR_calib.c')
endmember_name_list.append('CO2')

## Special case ZrSiO<sub>4</sub>

New component, not in previously published material.  
Properties are derived from Robie and Hemingway, 1995:  
Robie RA, Hemingway BS (1995) Thermodynamic Properties of Minerals and Related Substances at 298.15 K and 1 Bar (10<sup>5</sup> Pascals) Pressure and at Higher Temperatures. US Geological Survey Bulletin 2131, 461pp.

In [ ]:
model = coder.StdStateModel()
model.set_module_name('rMELTS_ZR')
model_type = "calib"
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

\$C_P = A_1 + A_2 T +  +  + A_5 T^2 \$

In [ ]:
h0,s0,v0,a1,a2,a3,a4,a5 = sym.symbols('h0 s0 v0 a1 a2 a3 a4 a5')
params = [('h0','J',h0), ('s0', 'J/K', s0), ('v0', 'J/bar', v0), 
          ('a1', 'J/K', a1), ('a2', 'J/K^2', a2), ('a3', 'J-K', a3), ('a4', 'J/K^1/2', a4), ('a5', 'J/K^3', a5)]
Cp = a1 + a2*T + a3/T**2 + a4/sym.sqrt(T) +a5*T**2
H = h0 + sym.integrate(Cp,(T,Tr,T))
S = s0 + sym.integrate(Cp/T,(T,Tr,T))
GPrP = sym.integrate(v0,(P,Pr,P))
G = H - T*S + GPrP
G

In [ ]:
model.add_expression_to_model(G, params)

In [ ]:
param_dict = {
    'Phase': 'ZrSiO4',
    'Formula': 'Zr(1)Si(1)O(4)',
    'h0':  -2034200.0,          # Robie and Hemingway
    's0':        84.0,          # Robie and Hemingway
    'v0':         3.926,        # Robie and Hemingway
    'a1':         2.370e2,      # Robie and Hemingway
    'a2':        -1.788e-2,     # Robie and Hemingway
    'a3':        -1.496e5,      # Robie and Hemingway
    'a4':        -2.268e3,      # Robie and Hemingway
    'a5':         0.0,          # Robie and Hemingway
    'T_r':      298.15,
    'P_r':        1.0
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat rMELTS_ZR.pyx >> endmembers.pyx
endmember_file_list.append('ZrSiO4_rMELTS_ZR_calib.c')
endmember_name_list.append('ZrSiO4')

## Import the module

First, update the cython build file to include all of the endmember modules

In [ ]:
with open('rMELTS_ZR.pyxbld', 'r') as f:
    fold = f.read()
    f.close()
old_file_list = "'" + endmember_file_list[-1] + "'"
new_file_list = ''
sep = ''
for x in endmember_file_list:
    new_file_list += sep + "'" + x + "'"
    sep = ", "
fnew = fold.replace(old_file_list, new_file_list)
with open('rMELTS_ZR.pyxbld', 'w') as f:
    f.write(fnew)
    f.close
%cp endmembers.pyx rMELTS_ZR.pyx

In [ ]:
import rMELTS_ZR
%cd ..

## Testing

Evaluate test functions at temperature (K) and pressure (bars)

In [ ]:
t = 1000.0
p = 1000.0

Generic testing method, utilized here and below

In [ ]:
from rubicon.objc import ObjCClass
src_cls = ObjCClass('LiquidMeltsPlusOldH2OandNewCO2')
src_obj = src_cls.alloc().init()

In [ ]:
def compare_model_and_timers(new_model='SiO2', old_model_index=0, print_results=True, time_g=True):
    def cmp_output(ref, x, y):
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:13.6e} {4:6.2f}%"
        if print_results:
            print(fmt.format(ref, x, y, x-y, 100.0*math.fabs((x-y)/y)))
        else:
            assert 100.0*math.fabs((x-y)/y) < 0.01, ref+' fails comparison!'
    def tst_output(ref, x):
        fmts = "{0:<10.10s} {1:13.6e}"
        if print_results:
            print(fmts.format(ref, x))
    refPhase = src_obj.endmembers[old_model_index]
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_g')(t,p)
    y = refPhase.getGibbsFreeEnergyFromT_andP_(t,p)
    cmp_output('G', x, y)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dgdt')(t,p)
    y = -refPhase.getEntropyFromT_andP_(t,p)
    cmp_output('dGdT', x, y)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dgdp')(t,p)
    y = refPhase.getVolumeFromT_andP_(t,p)
    cmp_output('dGdP', x, y) 
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdt2')(t,p)
    tst_output('d2GdT2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdtdp')(t,p)
    tst_output('d2GdTdP', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdp2')(t,p)
    tst_output('d2GdP2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdt3')(t,p)
    tst_output('d3GdT3', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdt2dp')(t,p)
    tst_output('d3GdT2dP', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdtdp2')(t,p)
    tst_output('d3GdTdP2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdp3')(t,p)
    tst_output('d3GdP3', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_s')(t,p)
    y = refPhase.getEntropyFromT_andP_(t,p)
    cmp_output('S', x, y)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_v')(t,p)
    y = refPhase.getVolumeFromT_andP_(t,p)
    cmp_output('V', x, y)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_cv')(t,p)
    tst_output('Cv', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_cp')(t,p)
    y = refPhase.getHeatCapacityFromT_andP_(t,p)
    cmp_output('Cp', x, y)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dcpdt')(t,p)
    tst_output('dCpdT', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_alpha')(t,p)
    tst_output('alpha', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_beta')(t,p)
    tst_output('beta', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_K')(t,p)
    tst_output('K', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_Kp')(t,p)
    tst_output('Kp', x)
    if time_g:
        print ('Time coder generated function ...')
        method = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_g')
        %timeit method(t, p)
        print ('Time provided Objective-C function ...')
        %timeit refPhase.getGibbsFreeEnergyFromT_andP_(t,p)

Test and time code

In [ ]:
for i,x in enumerate(endmember_name_list):
    if x != 'ZrSiO4':
        print ('Testing code for', x, '...')
        compare_model_and_timers(new_model=x, old_model_index=i, print_results=True, time_g=False)
        print ('')

In [ ]:
def output_model_and_timers(new_model='SiO2', print_results=True, time_g=True):
    def tst_output(ref, x):
        fmts = "{0:<10.10s} {1:13.6e}"
        if print_results:
            print(fmts.format(ref, x))
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_g')(t,p)
    tst_output('G', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dgdt')(t,p)
    tst_output('dGdT', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dgdp')(t,p)
    tst_output('dGdP', x) 
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdt2')(t,p)
    tst_output('d2GdT2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdtdp')(t,p)
    tst_output('d2GdTdP', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d2gdp2')(t,p)
    tst_output('d2GdP2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdt3')(t,p)
    tst_output('d3GdT3', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdt2dp')(t,p)
    tst_output('d3GdT2dP', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdtdp2')(t,p)
    tst_output('d3GdTdP2', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_d3gdp3')(t,p)
    tst_output('d3GdP3', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_s')(t,p)
    tst_output('S', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_v')(t,p)
    tst_output('V', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_cv')(t,p)
    tst_output('Cv', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_cp')(t,p)
    tst_output('Cp', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_dcpdt')(t,p)
    tst_output('dCpdT', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_alpha')(t,p)
    tst_output('alpha', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_beta')(t,p)
    tst_output('beta', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_K')(t,p)
    tst_output('K', x)
    x = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_Kp')(t,p)
    tst_output('Kp', x)
    if time_g:
        print ('Time coder generated function ...')
        method = getattr(rMELTS_ZR, 'cy_'+new_model+'_rMELTS_ZR_calib_g')
        %timeit method(t, p)

## Timing the ZrSiO4 code

In [ ]:
output_model_and_timers(new_model='ZrSiO4', print_results=True, time_g=True)